# Exploration and Debugging <br> 
### The objectives of this notebook are: 
* **Load** a sample data file from either the regular season or playoff;
* **Understand** an initial overview of the data structure;
* **Create** an ipywidget to interact with a few values of a particular game.

## Imports

In [1]:
import json
import numpy as np
import pandas as pd
import glob
import os
import matplotlib
import matplotlib.pyplot as plt

# FOR INTERACTIVE VISUALIZATION
import ipywidgets as widgets
from ipywidgets import interact, IntSlider, Dropdown, SelectionSlider
import ipywidgets as widgets
matplotlib.rcParams['figure.figsize'] = [10,5]
plt.style.use('fivethirtyeight')

## Data Loading

In [2]:
data_path = os.path.join(os.path.dirname(__file__), '..', 'ift6758','data', 'raw')

## Data Loading Functions

In [3]:
def decode_json_file(file_path) -> dict:
    """
    Decode the content of a single json file and returns the content as a dict.
    :param file_path: Path to json file
    :return: Parsed json file as dict
    """
    with open(file_path) as f:
        return json.load(f)

def read_json_file(season: str, is_playoff: bool, game_id: int,  data_path: str = '../ift6758/data/raw'):
    """
    Get and decode all JSON files path list corresponding to the season for reg or playoff matches
    :param season: 4 digit string representation of a season. ex: '2016'
    :param is_playoff: bool representing if match is in playoff or reg season
    :param path: raw data path
    :return: list of json files as dicts
    """
    reg_playoff_indicator = '03' if is_playoff else '02'
    f_path = os.path.join(data_path,f'{season}{reg_playoff_indicator}{game_id}.json')
    if os.path.exists(f_path):
        return decode_json_file(f_path)
    else:
        return "No Game"


## JSON Printing Functions

In [4]:
def pretty_print_game_info(json_file):
    """
    Pretty prints the general info of a game such as the teams participating and their scores
    :param json_file: json files as a dict
    :return:  None
    """
    if json_file.get('messageNumber',{}) == 2:
        return
    date = json_file['gameData']['datetime']['dateTime']
    single_game_id = str(json_file['gamePk'])[6::]
    home = json_file['gameData']['teams']['home']['abbreviation']
    away = json_file['gameData']['teams']['away']['abbreviation']
    home_score = ['Home',json_file['gameData']['teams']['home']['abbreviation'], json_file['liveData']['linescore']['teams']['home']['goals'],
     json_file['liveData']['linescore']['teams']['home']['shotsOnGoal'], json_file['liveData']['linescore']['teams']['home']['goaliePulled']]

    away_score = ['Away',json_file['gameData']['teams']['away']['abbreviation'], json_file['liveData']['linescore']['teams']['away']['goals'],
    json_file['liveData']['linescore']['teams']['away']['shotsOnGoal'], json_file['liveData']['linescore']['teams']['away']['goaliePulled']]
    score_df = pd.DataFrame(list(zip(home_score, away_score)),
               index =['', 'Teams','Goals','SoG','Goalie Pulled'],
               columns=['',''])
    print(date)
    print(f'Game ID: {single_game_id}')
    print(f'{home} (home) vs {away} (away)')
    print('\n----------Final Score----------')
    print(score_df)

def pretty_print_event_info(event):
    """
    Pretty prints the info of a particular event
    :param json_file: json files as a dict
    :return:  None
    """
    # Print all the event dictionary event that exist
    if 'about' in event.keys(): 
        print('{"about": { ')
        for key,value in event['about'].items():
            print(f'\t "{key}" : {value},')
        print('\t}')
    if 'players' in event.keys(): 
        print('{"players": { ')
        for p in event['players']:
            for key,value in p.items():
                print(f'\t "{key}" : {value},')
        print('\t}')
    if 'result' in event.keys(): 
        print('{"result": { ')
        for key,value in event['result'].items():
            print(f'\t "{key}" : {value},')
        print('\t}')
    if 'coordinates' in event.keys() and event['coordinates'] : 
        print('{"coordinates": { ')
        for key,value in event['coordinates'].items():
            print(f'\t "{key}" : {value},')
        print('\t}')
    if 'team' in event.keys(): 
        print('{"team": { ')
        for key,value in event['team'].items():
            print(f'\t "{key}" : {value},')
        print('\t}')
    print('}')


## Exploration of the data structure - Widget Interactions


In [5]:
@interact(
    # Dropdown reg/playoff season selector
    is_playoff=Dropdown(
        options=[('Regular', False), ('Playoff', True)],
        value=0,
        description='Match Type',
        continuous_update=False),
    # Season Selector **TODO match the season year standard across the project**
    season = IntSlider(
        value=2016,
        min=2016,
        max=2020,
        step=1,
        description='Season',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d')
    )
def display_season_and_isPlayoff(season, is_playoff):
    """
    Function responsible for the interactive slider controlling the season slider and the reg/playoff dropdown selector.
    Background image of the plot is also loaded here so as not to unecessarily reload the file on each update.
    :param season: string representing the season (ex: '2016')
    :param is_playoff: bool representing type of game (reg or playoff)
    :return:  None
    """
    reg_playoff_indicator = '03' if is_playoff else '02'
    paths = glob.glob(os.path.join(data_path,f'{season}{reg_playoff_indicator}*.json'))
    valid_game_id = sorted([s[-9:-5:] for s in paths])
    img = plt.imread("../figures/nhl_rink.png")
    
    @interact(
        game_id = SelectionSlider(
        options=valid_game_id,
        value=valid_game_id[0],
        description='Game-ID',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True)
    )
    def display_game_id(game_id):
        """
        Function responsible for the interactive slider controlling the game id slider. Pretty prints general game info.
        :param game_id: string representing the game-id (zero-filled) (ex: '0002')
        :return:  None
        """
        json_file = read_json_file(str(season),is_playoff,game_id)
        events = json_file.get('liveData', {}).get('plays', {}).get('allPlays', '')
            
        #events = json_file['liveData']['plays']['allPlays']
        event_ids = range(len(events))
        pretty_print_game_info(json_file)
        
        if(len(event_ids) > 0): 
            @interact(
            event_id = SelectionSlider(
            options= event_ids,
            value=0,
            description='Event ID',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True)
            )
            def display_event(event_id):
                """
                Function responsible for the interactive slider controlling the event id slider. List of event_ids is obtained dynamically based on what exists for that particular game.
                :param event_id: int representing the index of the event
                :return:  None
                """
                # Create plot
                
                # Display background image with correct ratio
                event = json_file['liveData']['plays']['allPlays'][event_id]
                # Build image title
                desc = event['result']['description']
                period_time = event['about']['periodTime']
                period = event['about']['period']
                title_string = f'{desc} \n {period_time} P-{period}'
                
                # Plot event coordinates if available
                if 'coordinates' in event.keys() and event['coordinates'] :
                    fig, ax = plt.subplots()
                    plt.title(title_string)
                    ax.imshow(img,extent=[-100, 100, -42.5, 42.5])
                    ax.plot(event['coordinates']['x'],event['coordinates']['y'],marker='o',markersize=18,color='green')
                    plt.show()
                pretty_print_event_info(event)
                 
                
            

    

interactive(children=(IntSlider(value=2016, continuous_update=False, description='Season', max=2020, min=2016)…